# Interfacing Directly with LLMs 
### (Taking the Web Interface Out of the Picture)

<img src="../images/llm_api_access.png" width="400">

In Part 1, we made our first API call using OpenRouter and saw how to connect to a model like DeepSeek using Python.
Now that you’ve made your first call to a model, let’s take a closer look at one of the most common ways to talk to modern LLMs: the `/chat/completions` endpoint.

Specfically in this section, you'll learn:
- What inputs this endpoint expects
- How the response is structured
- How it compares to using ChatGPT interactively
- Why this gives you more control and automation in research workflows

As a refresher, an API (Application Programming Interface) is a set of rules and protocols that allows different software applications to communicate and interact with each other. It acts as an intermediary, defining how one piece of software can request services or data from another. Behind the scenes almost any website, resource, or software you are interacting with on the internet will use and define an API to connect, talk to, and transfer data between resources. 

## Breaking down the `[POST] /chat/completions`?

When you use ChatGPT or similar tools, the conversational experience you have is powered by an API endpoint called chat/completions. This endpoint is the engine behind a continuous conversation. You send it a **history of messages**, and it responds with the **next message in the conversation**.

Normally, the service handles all of this for you. But in this workshop, we'll get into the specifics of how to build and manage these API requests yourself. 

### Building a ChatCompletions Request
When you send a request to the chat/completions endpoint, you're essentially providing the model with a list of messages. The model then generates a new message to add to that list.

Your request needs to include two main components: the model you want to use and the conversation history itself.

```json
{
    "model": "gpt-4.1",
    "messages": [
      {
        "role": "developer",
        "content": "You are a helpful assistant."
      },
      {
        "role": "user",
        "content": "Hello!"
      }
    ],
    ... optional parameters...
}
```
Let's break down the key parts of this request:
- `model` This specifies which large language model you want to use. You might choose a model like `gpt-4o` from OpenAI, `claude-3-opus` from Anthropic, or others.
- `messages` [This is a list of all the messages in the conversation so far. Each message object has two parts]
    - `role` [This defines who is "speaking." We will dive more into this later.]
    - `content` [This is the actual text of the message.] 

### Understanding the [`ChatCompletion`](https://platform.openai.com/docs/api-reference/chat/object) Response
Once you send a request, the API returns a ChatCompletion object. This object contains the model's new message, along with a lot of other useful information

```json
{
  "id": "chatcmpl-B9MHDbslfkBeAs8l4bebGdFOJ6PeG",
  "object": "chat.completion",
  "created": 1741570283,
  "model": "gpt-4o-2024-08-06",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hey, how are you?",
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1117,
    "completion_tokens": 46,
    "total_tokens": 1163,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": "fp_fc9f1d7035"
}

```

There's a lot of information here, but for now, you only need to focus on a few key parts:

- `choices` [This is an array that contains the model's response. In most cases, you'll just be looking at the first (and only) item in this list]
    - `message`: Inside choices, this is the new message object.
        - `role`: This will always be `assistant`, since it's the model's response.
        - `content`: This is the actual text generated by the model.

### Let's talk to our agent: A simple Chat Request

In [1]:
# First, setup our Client
from openai import OpenAI

# Read the API_KEY
with open('API_KEY.txt', 'r') as file:
    API_KEY = file.read()
    
# Intialize Client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1", 
  api_key=API_KEY,
)

Next, we'll build our first conversation request. This is where the `messages` array comes into play.

In [5]:
name = "Sohail"

completion = client.chat.completions.create(
  model="deepseek/deepseek-chat-v3-0324:free",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Hello! I am {<WHAT_GOES_HERE>}, nice to meet you!"} # Fill in your name
  ]
)

#### Instructor Notes: before you assume understanding and continue, stop here and ask people if they understand what just happened here. 

In [6]:
print(completion.choices[0].message.content)

Hello, Sohail! Nice to meet you too! 😊 How are you doing today? Is there anything I can help you with?


#### What's happening here?

The `client.chat.completions.create()` method builds and sends the API request for us.

- `model`: We're using a specific model available on OpenRouter, in this case, deepseek/deepseek-chat-v3-0324:free.
- `messages`: This is our conversation history. We start with a system message to set the model's persona and a user message with our initial prompt. The f-string f"Hello! I am {name}, nice to meet you!" is a neat way to dynamically insert variables into your messages.
- `print(completion.choices[0].message.content)`: This line shows how to parse the JSON response we talked about earlier. We access the first item in the choices list, then the message object, and finally the content field to get the text of the model's reply.

## Simulating Memory with Message History
A common misconception is that LLMs "remember" previous interactions. They don't. Each API request is entirely stateless, the model only knows what's in the messages list you send.

To build a continuous conversation, you must simulate memory by including all prior messages in every new API request. Let's see this in action.

#### Example 1: Ask the Model To Remember Your Name
In this first example, we include the entire conversation history in our request, so the model knows the user's name.

In [16]:
# Ask the model to remember your name
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "My name is Sohail."},
    {"role": "user", "content": "What is my name?"}
]

completion = client.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=messages
)

In [17]:
print(completion.choices[0].message.content)

Your name is **Sohail**. 😊 Let me know if there's anything I can assist you with, Sohail!


#### Example 2: The Model Forgets
Now, what happens if we only send the final message? This is like starting a new chat in ChatGPT which has no history of your previous isolated conversations. 

In [19]:
# A new, isolated conversation history
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is my name?"}
]

# Send the request with only the last message
completion = client.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=messages
)

# This will likely respond with something like "I don't know your name."
print(completion.choices[0].message.content)

I don’t have access to personal information about you, including your name, unless you share it with me. Let me know how I can assist you, and I’ll be happy to help! 😊


---
As you can see, without the "My name is Sohail" message, the model has no context. You are **responsible for managing** this message history yourself.

💡
LLMs don’t have memory by default. If you want the model to remember something, you have to simulate memory by including prior messages in the messages list. In a platform like ChatGPT, this happens automatically: the UI handles the conversation history behind the scenes. But when working directly with the API, you’re in charge of preserving that history yourself.

### A Deeper Look at the Roles: `user`, `assistant`, and `system`

By now, you've seen our messages list contain objects with different role values. The role is a crucial part of the API request; it tells the model who is speaking and provides essential context for its response.

The three primary roles you will use are:
- `user`: This represents **your** input - the prompt, the data, or the question you're giving the model. In our thematic coding scenario, for example, each essential worker's story would be placed within a user message.
- `assistant`: This is the model's response. When the model generates a reply, its role is assistant. When building a multi-turn conversation, you'll take the model's response and add it back to the messages list with this role to preserve the conversation history.
- `system`: This is a special, high-level role used to set the model's overall behavior, persona, or instructions before the conversation begins. Unlike user and assistant messages, the system message is not part of the back-and-forth chat; rather, it's a foundational set of rules that the model should follow throughout the entire interaction.

Understanding these roles is key to building effective and reliable API calls. The system role, in particular, is an extremely powerful tool for a researcher.

---
Because the system message has a privileged position and often carries more weight than user messages, it is your primary tool for "prompt engineering" at a global level.

A key benefit of the system role is its ability to enforce constraints and rules, ensuring consistent behavior across many data points. This is crucial for a research task like thematic coding.

Let's use our COVID-19 narrative scenario to demonstrate this. We can use the system prompt to provide a strict set of instructions and a codebook for the model to follow.

In [4]:
research_prompt = """
You are an expert qualitative researcher assisting with a study on COVID-19 narratives.
Your task is to analyze the following story and extract specific themes and details.

Follow these rules precisely:
1.  Identify the emotions expressed in the story.
2.  Note any mentions of material conditions (e.g., shortages of PPE, crowded spaces).
3.  Identify themes of solidarity or isolation.
4.  Do not add any additional text or explanation outside of the JSON object.
"""

story = """
We ran out of masks again. There were nights when I cried the whole subway ride home, not because I was scared—though I was—but because I felt like no one saw us.
"""

# The system message provides the instructions and codebook
messages = [
    {"role": "system", "content": research_prompt},
    {"role": "user", "content": story}
]

completion = client.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=messages,
    response_format={"type": "json_object"}
)

# The model's response will be a clean JSON object, ready for your analysis
print(completion.choices[0].message.content)

{
  "emotions": ["fear", "sadness", "invisibility"],
  "material_conditions": ["shortage of masks", "crowded subway"],
  "solidarity_or_isolation": "isolation"
}


Note: In this example, we've also introduced an optional parameter, `response_format={"type": "json_object"}`. This is a powerful feature that instructs the model to only return valid JSON, which is essential for a repeatable, programmatic workflow.

This demonstrates how a strong system prompt can transform a general-purpose model into a specialized research assistant, ensuring that every request returns a consistent, structured output that you can easily process and analyze at scale.

### [Challenge]: Let's build a Multi-Turn Conversation Loop
Now that you understand how to use the messages list to provide context, we can build a dynamic conversation. Instead of manually creating a new messages list for each turn, we will create a simple loop that appends the user's new message and the model's new response to the conversation history.

#### 🥊 **Challenge 1**: Continue the Conversation

In [36]:
def get_completions_response(message_history, new_user_message):

    messages = message_history

    """
        Hint: 
        We need to append our new user message to messages. Think about the most important parameters that go into a message request. 
    """
    messages.append(
        # {
        #     #... Fill this in, what 2 parameters are essential in a message request body?
        #     "role": "<REPLACE>",
        #     "...": "<REPLACE>"
        # }

        {"role": "user", "content": new_user_message}
    )

    model = "deepseek/deepseek-chat-v3-0324:free"
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )

    return response, messages

In [38]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Classify the following tweets about the new city congestion tax into one of three categories: In Favor, Against, Neutral."},
    {
        "role": "user", 
        "content": """
            Tweets to classify:
                "Finally! The congestion tax will reduce traffic and make downtown air cleaner. About time the city acted!"
                "This tax is just another way for politicians to squeeze money out of regular people. Terrible idea."
                "Not sure yet how the congestion tax will affect my commute. Need more info before I decide."
                "Public transport is already overcrowded, and now they want to charge us more to drive. Ridiculous."
                "Great move! Other cities did this and saw big drops in pollution.
        """
    },
    {
        "role": "assistant",
        "content": """Here’s the classification of the tweets:  \n\n1. **"Finally! The congestion tax will reduce traffic and make downtown air cleaner. About time the city acted!"**  \n   → **In Favor** (Positive sentiment, supports the policy)  \n\n2. **"This tax is just another way for politicians to squeeze money out of regular people. Terrible idea."**  \n   → **Against** (Negative sentiment, criticizes the policy)  \n\n3. **"Not sure yet how the congestion tax will affect my commute. Need more info before I decide."**  \n   → **Neutral** (Uncertainty, neither support nor opposition)  \n\n4. **"Public transport is already overcrowded, and now they want to charge us more to drive. Ridiculous."**  \n   → **Against** (Negative sentiment, opposes the tax)  \n\n5. **"Great move! Other cities did this and saw big drops in pollution."**  \n   → **In Favor** (Positive sentiment, highlights benefits of the policy)  \n\nLet me know if you\'d like further refinements!"""
    }
]

response, messages = get_completions_response(messages, "Can you summarize the results for me?")
print(response)

ChatCompletion(id='gen-1755483217-AJ1sB7l5UqlIf5fXHl9n', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Certainly! Here's a summary of the sentiment classification for the tweets:  \n\n- **In Favor (Positive)**: 2 tweets (pollution reduction, traffic improvement)  \n- **Against (Negative)**: 2 tweets (seen as a money grab, burdens drivers)  \n- **Neutral (Undecided)**: 1 tweet (needs more information)  \n\n### Breakdown:  \n- The positive tweets highlight environmental and traffic benefits.  \n- The negative tweets focus on financial strain and poor public transport conditions.  \n- Only one user remains neutral, expressing uncertainty.  \n\nWould you like any further analysis?", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1755483217, model='deepseek/deepseek-chat-v3-0324:free', object='chat.completion', service_tier=None, 

In [39]:
def update_message_history_with_response(message_history, completion_response):
    # Let's update our running message history with the response we just got back from the response.
    # Stop here and ask yourself... why are we doing this?

    # Hint: Look at the ChatCompletion Object. We just need to extract a specific object from this element
    # There's two ways to do this, manually creating a JSON object or using the object directly from the ChatCompletion Object itself. 
    message_history.append(completion_response.choices[0].message)

    return message_history

In [40]:
from pprint import pprint
pprint(messages)

[{'content': 'You are a helpful assistant.', 'role': 'system'},
 {'content': 'Classify the following tweets about the new city congestion tax '
             'into one of three categories: In Favor, Against, Neutral.',
  'role': 'user'},
 {'content': '\n'
             '            Tweets to classify:\n'
             '                "Finally! The congestion tax will reduce traffic '
             'and make downtown air cleaner. About time the city acted!"\n'
             '                "This tax is just another way for politicians to '
             'squeeze money out of regular people. Terrible idea."\n'
             '                "Not sure yet how the congestion tax will affect '
             'my commute. Need more info before I decide."\n'
             '                "Public transport is already overcrowded, and '
             'now they want to charge us more to drive. Ridiculous."\n'
             '                "Great move! Other cities did this and saw big '
             'dr

In [30]:
messages = update_message_history_with_response(messages, response)
pprint(messages)

[{'content': 'You are a helpful assistant.', 'role': 'system'},
 {'content': 'Classify the following tweets about the new city congestion tax '
             'into one of three categories: In Favor, Against, Neutral.',
  'role': 'user'},
 {'content': '\n'
             '            Tweets to classify:\n'
             '                "Finally! The congestion tax will reduce traffic '
             'and make downtown air cleaner. About time the city acted!"\n'
             '                "This tax is just another way for politicians to '
             'squeeze money out of regular people. Terrible idea."\n'
             '                "Not sure yet how the congestion tax will affect '
             'my commute. Need more info before I decide."\n'
             '                "Public transport is already overcrowded, and '
             'now they want to charge us more to drive. Ridiculous."\n'
             '                "Great move! Other cities did this and saw big '
             'dr

#### Challenge 2: Put this together to build a chat conversation tool like ChatGPT that will remember what you say in history

In [43]:
message_history = [
    {"role": "system", "content": "You are a helpful assistant."}
]
while True:
    user_input = input("Ask anything... (or 'quit' to exit): ")
    if user_input.lower() == 'quit':
        break
        
    response, messages = get_completions_response(message_history, user_input)
    print(response.choices[0].message.content)
    message_history = messages
    update_message_history_with_response(message_history, response)

Ask anything... (or 'quit' to exit):  hi


Hello! How can I assist you today? 😊


Ask anything... (or 'quit' to exit):  what did i just say?


You just said: *"hi"*  

And before that, you asked: *"what did i just say?"*  

I’m happy to help with anything you need—just let me know! 😊


Ask anything... (or 'quit' to exit):  quit


# Zero-Shot vs. Few-Shot Prompting

Now that you have a firm grasp of the API's mechanics, let's explore how to get the model to produce the exact output you need. This is where the practice of prompting comes in.

We'll return to our COVID-19 scenario and the task of thematic coding. We want the model to act as a human coder, identifying specific themes from the essential workers' narratives.

## Zero-Shot Prompting: "Just Ask For It"

Zero-shot prompting is when you ask the model to perform a task without giving it any examples. You rely entirely on the model's pre-trained knowledge to understand your request.

Let's try this with one of our narratives. We will give the model the story and simply ask it to extract the themes, without any examples or explicit instructions on the output format.

💡 Tip: In most cases, you don't have to think too much about the system prompt. Keeping it at a short and simple `{"role": "system", "content": "You are a helpful assistant."}` will suffice. 

In [11]:
# A new narrative from an essential worker
narrative = """
People think of front‑line workers…the grocery workers, transit workers, the first responders… as having helped the city get through it. But that’s not what happened. We helped the city survive it.
"""

# The messages list for our Zero-Shot request
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Analyze the following story and extract key themes: {narrative}"}
]

In [12]:
# Send the request
completion = client.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=messages
)

print(completion.choices[0].message.content)

The story you've shared is a poignant reflection on the role of front-line workers during a crisis, likely referencing the COVID-19 pandemic or a similar societal challenge. Here are the key themes extracted from the passage:

1. **Essential Labor and Survival**: The narrative highlights the critical role of front-line workers (grocery workers, transit workers, first responders) in ensuring the literal survival of the city during a crisis. It shifts the focus from mere "getting through" a hardship to emphasizing their role in sustaining life and basic societal functions.

2. **Underappreciation and Misperception**: The statement challenges a common perception—that these workers merely "helped the city get through it"—by asserting a deeper truth: their labor was foundational to survival. This suggests a frustration with how their contributions might be minimized or oversimplified by society.

3. **Sacrifice and Resilience**: The tone implies that front-line workers bore a disproportiona

---
The model will likely respond with a good analysis, but the format will be inconsistent. It might be a list, a paragraph, or a different style each time, which makes it very difficult to process programmatically.

## Few-Shot Prompting: Let's give it some examples to show it what we want

Few-shot prompting is when you give the model one or more examples of the input and the desired output. By showing it what you want, you significantly increase the likelihood of getting a consistent, structured response.

This approach is like giving a new researcher a codebook with a few pre-coded examples to learn from.

At the simplest level, you can think of few-shot prompting as giving pointers or guidance to the LLM on how to respond. It's similar to how you might teach someone a new task by showing them a few clear examples rather than just describing the process. This approach is invaluable for getting consistent, structured output from the model.

Let's see this in action with a few simple scenarios.

### Scenario 1: Thematic Identification

Imagine you want to identify the main theme of a short story. A simple instruction (zero-shot) might not give you the format you need.

* **Zero-Shot Prompt:** "What is the theme of this story about a detective who solves a case using a seemingly insignificant detail?"
* **Likely Zero-Shot Response:** "The theme of the story is attention to detail, as the detective's success hinges on a small, overlooked fact."
This is a good response, but what if you wanted it to be just a single word? Let's use few-shot prompting to guide it.

* **Few-Shot Prompt:**
    * **Input 1:** "A young woman moves to a new city and learns to navigate a demanding job and a new social circle."
    * **Output 1:** "Theme: Coming-of-age"
    * **Input 2:** "Two rival knights must team up to defeat a dragon that is terrorizing their kingdoms."
    * **Output 2:** "Theme: Collaboration"
    * **Input 3:** "A story about a detective who solves a case using a seemingly insignificant detail."
* **Likely Few-Shot Response:** "Theme: Attention to detail"
By giving just two examples, we've taught the model the exact output format we want: Theme: [single word].

### Scenario 2: Simple Data Extraction

What if you need to extract specific information, like names and dates?

* **Zero-Shot Prompt:** "Find the person's name and birthday in this sentence: 'The grand opening, attended by CEO Jane Doe, was held on October 26, 2024, to celebrate her 45th birthday.'"
* **Likely Zero-Shot Response:** "The person's name is Jane Doe and her 45th birthday is on October 26, 2024."
Again, the response is correct, but not in a format you can easily use in a spreadsheet or database. Now let's try a few-shot prompt to get a clean, structured list.

* **Few-Shot Prompt:**
    * **Input 1:** "The presentation was given by Dr. Alan Turing on June 23, 1912."
    * **Output 1:** "Name: Alan Turing, Date: June 23, 1912"
    * **Input 2:** "Marie Curie's discovery on November 7, 1867, changed the world."
    * **Output 2:** "Name: Marie Curie, Date: November 7, 1867"
    * **Input 3:** "The grand opening, attended by CEO Jane Doe, was held on October 26, 2024, to celebrate her 45th birthday."
* **Likely Few-Shot Response:** "Name: Jane Doe, Date: October 26, 2024"

### Scenario 3: Abstract Text Analysis

Few-shot prompting can even be used for more abstract tasks, like determining a character's alignment based on their actions, a common task in fantasy or game-related analysis.

* **Zero-Shot Prompt:** "Is a character who regularly disobeys laws to help others and who values personal freedom over societal order good, evil, or neutral?"
* **Likely Zero-Shot Response:** "A character who regularly disobeys laws to help others and values personal freedom over societal order could be considered a chaotic good character in many alignment systems."
This is a good, detailed answer, but what if you want a simpler classification? We can provide examples of what "lawful," "chaotic," "good," and "evil" mean in practice to guide the model's output.

* **Few-Shot Prompt:**
    * **Input 1:** "A character who robs a corrupt merchant to give gold to the poor, believing personal conscience is more important than law."
    * **Output 1:** "Alignment: Chaotic Good"
    * **Input 2:** "A character who follows every law and rule to the letter, even if it leads to a bad outcome."
    * **Output 2:** "Alignment: Lawful Neutral"
    * **Input 3:** "A character who regularly disobeys laws to help others and who values personal freedom over societal order."
* **Likely Few-Shot Response:** "Alignment: Chaotic Good"
As you can see, the model learns the exact structure from the examples and applies it to the new input, making the output predictable and machine-readable. This simple technique is the foundation for getting the clean JSON responses we need for our research workflow.

### Few Shot Examples with JSON

Let's go back to our coding task. We want the output to be a clean JSON object with specific keys. We can provide the model with an example of a coded narrative to guide its response.

In [13]:
# A new narrative for the model to analyze
narrative_2 = """
We ran out of masks again. There were nights when I cried the whole subway ride home, not because I was scared—though I was—but because I felt like no one saw us.
"""

# The message containing the example
example_prompt = """
Here is an example of a coded narrative and its desired JSON output:

**Narrative:**
People think of front‑line workers…the grocery workers, transit workers, the first responders… as having helped the city get through it. But that’s not what happened. We helped the city survive it.

**Coded JSON Output:**
{
  "emotion": ["anger", "resilience"],
  "material_conditions": ["none"],
  "solidarity": "absent",
  "theme": "invisibility of labor"
}

Now, please code the following narrative using the same format.
"""

# The messages list for our Few-Shot request
messages = [
    {"role": "system", "content": "You are an expert qualitative researcher."},
    {"role": "user", "content": f"{example_prompt}\n\n**Narrative to code:**\n{narrative_2}"}
]

# Send the request with the example
completion = client.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=messages,
    response_format={"type": "json_object"}
)

print(completion.choices[0].message.content)

# The model is now much more likely to respond with a valid JSON object.

{
  "emotion": ["fear", "sadness", "frustration"],
  "material_conditions": ["lack of resources"],
  "solidarity": "absent",
  "theme": "invisibility of labor"
}


---
Key Takeaways
- Zero-shot is great for simple, general tasks, but it lacks control over the output format.
- Few-shot is your best friend when you need the model to follow a specific format or style. The examples you provide are crucial for guiding the model's response and ensuring consistency, which is vital for programmatic analysis.

By combining few-shot prompting with a powerful system prompt and the response_format parameter, you can build a highly reliable and scalable data extraction tool for your research.

Providing examples seemed to help the model return data in a format, more closely related to what we want. But notice, it still adds surrounding context and follow up questions. 

🔔 Question: Why would this response still be suboptimal if you were a researcher trying to extract information at scale?

## Structured Output: Guaranteed Responses

Structred output refers to the ability of the `ChatCompletions` API to return responses in a predefined format, such as a JSON object or a Pydantic Model. This is particulay useful when you need the model to adhere to a specific schema for downstram processing or integration with other systems. By defining the expected structure, you can ensure the response is validated and parsed into a predictable format. 

Key Features of Structured Outputs

1. Customizable Response Format
    - You can specify the expected structure of the response using the response_format parameter.
    - This can be defined as either a JSON schema or a Pydantic model, depending on your requirements.
2. Using JSON Schema with create:
    - The `chat.completions.create` method allows you to provide a JSON schema via the `response_format` paramater.
    - This guides the model to generate responses in the desired structure without requiring Python-based schema definitions.
3. Using Pydantic Models with parse
    - The `chat.completions.parse` method supports validation and parsing using Pydantic models.
    - This is ideal for scenarios where you need Python-based schema definitions and strict adherance to the structure.

### Setting up Structured Output

In [65]:
from pydantic import BaseModel

# Define the expected structure of the response
class ParsedSentence(BaseModel):
    subject: str
    verb: str
    obj: str

In [71]:
# Make the request to extract parts of a simple sentence
response = client.chat.completions.parse(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[
        {
            "role": "system",
            "content": "Extract the grammatical components from the sentence.",
        },
        {"role": "user", "content": "The cat chased the mouse."},
    ],
    response_format=ParsedSentence,
)

In [89]:
print(response.choices[0].message)

ParsedChatCompletionMessage[ParsedSentence](content='{  \n  "subject": "The cat",  \n  "verb": "chased",  \n  "obj": "the mouse"  \n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, parsed=ParsedSentence(subject='The cat', verb='chased', obj='the mouse'), reasoning=None)


🔔 Question: How can we extract our parsed message from this `ParsedChatCompletionMessage` Object? What fields can you see?

In [90]:
# To extract our Structured Response
print(response.choices[0].message.parsed)

print("Subject:", response.choices[0].message.parsed.subject)
print("Verb:", response.choices[0].message.parsed.verb)
print("Obj:", response.choices[0].message.parsed.obj)

subject='The cat' verb='chased' obj='the mouse'
Subject: The cat
Verb: chased
Obj: the mouse


### 🥊 [Final Challenge]: Putting it all togther - Classification using Structured Output

The goal of this final challenge is to combine everything you've learned to build a reliable, end-to-end workflow for structured data extraction. You will take a new narrative from an essential worker and use a Pydantic model to extract key themes in a validated, structured format.

Your Goal:
Using the DeepSeek API, your task is to create a Python script that takes the provided new_narrative as input and produces the expected_output by:
1. **Defining a Pydantic Model**: Create a BaseModel that accurately represents the structure of the expected_output.
2. **Building the Prompt**: Construct a messages list that includes the system prompt, a few-shot example, and the final user message containing the new_narrative.
3. **Making the API Call**: Use the client.chat.completions.create method to call the DeepSeek model.
4. **Validating the Response**: Use your Pydantic model to validate and parse the raw JSON string returned by the LLM.

#### The Input

This is the narrative you will be using as input for the LLM.

```python
new_narrative = """
In the quiet of the night, I'd mop floors at the hospital, the only sound the soft swish of the bucket. Patients came and went, doctors hurried past. Sometimes, they'd look right through me. But I'd always tell myself: someone has to keep this place clean for the healers to do their healing. That thought got me through the loneliest shifts.
"""

#### The Expected Output
Your script should produce a Pydantic object that, when printed, looks like this. This is your target.

```python
ThematicAnalysis(emotion=['loneliness', 'dedication'], material_conditions=['hospital cleaning'], solidarity='present', theme='invisibility of labor and pride')```

Hint: You will need to define the ThematicAnalysis Pydantic class to have fields for emotion, material_conditions, solidarity, and theme, just like the previous examples. Be sure to use the correct data types and a Literal type for the solidarity field. Good luck!

In [ ]:
import json
from openai import OpenAI
from pydantic import BaseModel
from typing import List, Literal, Optional

# (Assuming client is already initialized)
client = OpenAI(
    api_key="...",  # Replace with your API key
    base_url="[https://api.deepseek.com/v1](https://api.deepseek.com/v1)"
)

# The new narrative to analyze
new_narrative = """
In the quiet of the night, I'd mop floors at the hospital, the only sound the soft swish of the bucket. Patients came and went, doctors hurried past. Sometimes, they'd look right through me. But I'd always tell myself: someone has to keep this place clean for the healers to do their healing. That thought got me through the loneliest shifts.
"""

# Step 1: Define the Pydantic class here to match the desired output structure.
# Hint: Your class should have fields for 'emotion', 'material_conditions', 'solidarity', and 'theme'.
#       Remember to use the correct data types!
#
#       You may need to look up how to allow list datatypes in Pydantic. 
# class ThematicAnalysis(...):
#     ...



# Step 2: Build the few-shot prompt.
# Hint: The messages list needs a system prompt, a user/assistant example pair,
#       and the final user message with the new narrative.
#
# messages = [
#     ...
# ]


# Step 3: Make the API call to the DeepSeek model using `.parse`.
# Hint: The `.parse` method handles the validation and returns a Pydantic object directly.
#
# parsed_analysis = client.chat.completions.parse(
#     model="deepseek/deepseek-chat-v3-0324:free",
#     messages=messages,
#     response_format=...  # Specify your Pydantic class here
# )


# # The `parsed_analysis` variable now holds a validated Pydantic object!
# print(parsed_analysis)

## Conclusion: What We've Learned

In this workshop, you've moved from the conceptual understanding of LLM APIs to hands-on, programmatic control. You now have the foundational knowledge to build powerful, automated workflows with language models.

Here’s a quick summary of the key concepts you’ve mastered:

1.  **API Mechanics:** You understand that LLM APIs are **stateless**. To simulate memory and maintain context, you are **responsible for managing the message history** yourself by sending a full list of messages with each new request.
2.  **The Power of Roles:** You know how to use the `system`, `user`, and `assistant` roles to give the model instructions, provide it with your prompts, and capture its responses. The `system` role is particularly powerful for setting high-level rules and persona.
3.  **Zero-Shot vs. Few-Shot Prompting:** You've seen how **zero-shot** prompting is great for general tasks but can result in inconsistent output. In contrast, **few-shot** prompting is essential for guiding the model to produce a consistent, predictable format by providing it with a single example or a few examples.
4.  **Structured Output:** You now have the ultimate tool for reliability: **structured output with Pydantic**. By defining a `BaseModel`, you can give the LLM a clear blueprint for its response, and the `.parse` method ensures the output is always a valid, usable Python object. This is the critical step for moving from simple chat to scalable data analysis.

The specific example we worked through of thematic coding of a narrative is just one use case. The most powerful concept you’ve learned today is the idea of using structured output to programmatically define how the LLM should respond. This is a foundational technique that you can apply across all social science workflows, whether you are doing **data extraction**, **summarization**, or **classification**. By providing the LLM with a schema, you gain precise control over its output, making it a reliable and powerful tool for your research.